# Extended Dynamic Mode Decomposition on Limit Cycle

In this tutorial, we explore the (Extended-) Dynamic Mode Decomposition (E-DMD). We set up a non-linear ordinary differential equation (ODE) system, generate time series data with it and model the dynamics with an `EDMD` model. 

Note that all models for time series modelling require a `TSCDataFrame` type to fit a model. The initial conditions for the `predict` method can be either a `numpy.ndarray`, a `pandas.DataFrame`, or in some circumstances (when multiple samples are required to define an initial condition) a `TSCDataFrame`.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from datafold import (
    EDMD,
    DMDStandard,
    GaussianKernel,
    TSCPolynomialFeatures,
    TSCRadialBasis,
)
from datafold.utils._systems import Hopf
from datafold.utils.general import generate_2d_regular_mesh

## Set up Hopf ODE system

We set up a Hopf system:

$$
\dot{y}_0 = -y_1 + y_0 (\mu - y_0^2 - y_1^2)
$$
$$
\dot{y}_1 = y_0 + y_1 (\mu - y_0^2 - y_1^2)
$$

with $\mu=1$. The ODE system has an circle shaped attractor which is centered at the origin. All sampled initial conditions are off the attractor (i.e. the time series are sampled on the transient phase space region). 

We solve the system by integration with a Runge-Kutta45 scheme using scipy's ODE solver. The return type of the method function is a `TSCDataFrame` and includes the time series for each initial condition (a row in argument `X_ic`).

## Sampling the dynamical system

We now start collecting time series data from the Hopf system (our training set). To sample the phase space, we systematically distribute initial conditions and solve the ODE system for rather short time intervals.

In [ ]:
system = Hopf()

# set up training data by sampling original Hopf system
n_timesteps = 21
time_values = np.linspace(0, 0.4, n_timesteps)

X_ic = generate_2d_regular_mesh(
    low=(-2, -2),
    high=(2, 2),
    n_xvalues=8,
    n_yvalues=8,
    feature_names=system.feature_names_in_,
)
X_tsc = system.predict(X_ic, time_values=time_values)

print(f"time delta: {X_tsc.delta_time}")
print(f"nr. time series: {X_tsc.n_timeseries}")
print(f"nr. timesteps per time series: {X_tsc.n_timesteps}")
print(f"(n_samples, n_features): {X_tsc.shape}")
print(f"time interval {X_tsc.time_interval()}")
print(f"Same time values: {X_tsc.is_same_time_values()}")
print("")
print("Data snippet fo training data:")
X_tsc

### Plot: Sampled time series used for training

In [ ]:
# function to add a single arrow in the following time series plots
idx_arrow = np.array([time_values.shape[0] // 2 - 1, time_values.shape[0] // 2])

def include_arrow(ax, df):
    arrow = df.iloc[idx_arrow, :]
    ax.arrow(
        arrow.iloc[0, 0],
        arrow.iloc[0, 1],
        dx=arrow.iloc[1, 0] - arrow.iloc[0, 0],
        dy=arrow.iloc[1, 1] - arrow.iloc[0, 1],
        color="black",
        head_width=0.05,
    )

In [ ]:
fig, ax = plt.subplots(figsize=[5, 5])

for _id, df in X_tsc.itertimeseries():
    ax.plot(df["x1"].to_numpy(), df["x2"].to_numpy(), 0.1, c="black")
    include_arrow(ax, df)

ax.set_title("sampled time series data from ODE system")
ax.set_xlabel("x1")
ax.set_ylabel("x2")
ax.axis("equal")
ax.grid();

## 1. DMD: Identity dictionary

In our first model, we use a Dynamic Mode Decomposition (located in `datafold.dynfold.dmd`) model and decompose the data in spatio-temporal coordinates using the original form of the time series. In other words, our dictionary only includes the state identities "x1" and "x2" as observable functions.

In the first attempt, we use `DMDStandard` directly. The same could be accomplished with `EDMD(dict_step=["id", TSCIdentity()]`).

Note that the DMD-based models' API aligns with scikit-learn. However, the input type of `X` is restricted to a `TSCDataFrame`. The `predict` method allows setting an array of `time_values`, where we can choose at which time samples to evaluate the model. In our case, we are interested in reconstructing the training data and set the prediction to the same time values as in `X` during `fit`.

In [ ]:
dmd = DMDStandard().fit(
    X=X_tsc, store_system_matrix=True
)  # X must be of type TSCDataFrame
dmd_values = dmd.predict(X_tsc.initial_states(), time_values=X_tsc.time_values())

# perform out-of-sample prediction (not contained in training data)
# note that the time sampling are independent of the original samples
dmd_values_oos = dmd.predict(np.array([-1.8, 2]), time_values=np.linspace(0, 100, 1000))

print(
    "Data snipped for predicted time series training data and out-of-sample prediction"
)
dmd_values

In [ ]:
dmd_values_oos

### Compare with training data 

We can now compare the original time series data with the data-driven reconstruction of the DMD model. From what we see in the plots below is that the DMD model performs poorly. This is not surprising at this stage, because we learn the Koopman matrix directly on the available states. The computed Koopman matrix is therefore a $K \in \mathbb{R}^{[2 \times 2]}$ describing a linear system

$$ x_{n+1} = K x_n $$

and not being able to desribe a complex dynamics such as this of the underlying system. Note that the learnt system equation implies that we have modelled a dicrete system, while the underling system is continuous. This is a result from the discretely sampled data with a fixed time interval. Because we are in this easier setting of a 2-by-2 matrix, in the next cell, we look at the relation to a continuous system.

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(14, 5))
for _id, df in X_tsc.itertimeseries():
    ax[0].plot(df["x1"].to_numpy(), df["x2"].to_numpy(), 0.1, c="black")
    include_arrow(ax[0], df)

ax[0].set_title("training data used during fit")
ax[0].set_xlabel("x1")
ax[0].set_ylabel("x2")
ax[0].axis("equal")
ax[0].grid()

for _id, df in dmd_values.itertimeseries():
    ax[1].plot(df["x1"].to_numpy(), df["x2"].to_numpy(), 0.1, c="black")
    include_arrow(ax[1], df)

ax[1].set_title("DMD model (identity state dictionary)")
ax[1].set_xlabel("x1")
ax[1].set_ylabel("x2")
ax[1].axis("equal")
ax[1].grid()


# generate red "out-of-sample" prediction, for extra analysis below
ax[1].plot(
    dmd_values_oos["x1"].to_numpy(),
    dmd_values_oos["x2"].to_numpy(),
    0.1,
    c="red",
    linewidth=3,
)
include_arrow(ax[1], dmd_values_oos)

### Connection of discrete and continuous linear dynamical system

We start with the form of a continuous linear dynamical system

$$ \frac{dx}{dt} = A x $$

where $A$ is a constant system matrix. To connect this representation to our learnt discrete system ($x_{n+1} = K x_n$), we first discretize the time derivative on the left hand side with a usual forward finite difference

$$\frac{x_{n+1} - x_{n}}{\tau} = A x_n$$

where we set $\tau$ to the time sampling interval of the available time series data (in the data from the Hopf system $\tau = 0.02$). This is again a discrete system; then by rearranging the equation to the future state $x_{n+1}$ reveals

$$x_{n+1} = \underbrace{(I + \tau \cdot A)}_K x_n$$

We can then analyze both system matrices -- either in terms of a discrete system $K$ or via the continuous system matrix $A$.

$$A = \frac{K - I}{\tau}$$

Because we are still in the case of $A \in \mathbb{R}^{[2 \times 2]}$, where the observable states to approximate $K$ directly match the original states, we can now look at the the phase portrait and apply [stability theory](https://en.wikipedia.org/wiki/Stability_theory). With the computed values below, $\operatorname{trace}(A)$ and $\operatorname{det}{A}$, we can classify our current system as a "spiral sink" -- the determinant is larger than $\Delta$ and the trace is negative; see Poincaré diagram in the stability theory article. This matches the fast decaying red trajectory in the plot above.

Because the underlying Hopf system, that we aim to model, has quite different dynamics than a spiral sink, we continue our tutorial to improve the quality of our model.

In [ ]:
generator_A = (dmd.system_matrix_ - np.eye(2)) / dmd.dt_

det = np.linalg.det(generator_A)
trace = np.trace(generator_A)

print("Relevant values for the stability analysis: \n")
print(f"determinant of A: {det}")
print(f"trace of A: {trace}")

print(f"Delta {1/4. * trace ** 2} ")

## 2. EDMD: Polynomial feature dictionary

We now get to the "extended" part of a Dynamic Model Decomposition: We define a *dictionary* in which we process the time series data before we fit a DMD model with it. For this, we use the `datafold.appfold.EDMD` class, which is a [`sklearn.pipeline.Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html?highlight=pipeline#sklearn.pipeline.Pipeline). In the `EDMD` model, a dictionary can be a flexible number of transform models that are process the time series data consecutively (in the same order as defined). The final estimator has to be a `datafold.dynfold.dmd.DMDBase` model and defaults to `DMDFull`.  

Choosing the "right" dictionary is not an easy task and is similar to "model selection" in classical machine learning. In our choice of dictionary, we can include expert knowledge, e.g. if we know the principle equations from an underlying physical system from which time series are collected. We can also apply methods from functional theory to represent the data in another basis to linearize the unknown dynamics manifold. 

In the first dictionary, we use `TSCPolynomialFeatures` which is a wrapper of [`sklearn.preprocessing.PolynomialFeatures`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html?highlight=polynomial#sklearn.preprocessing.PolynomialFeatures) to support `TSCDataFrame` type.

In [ ]:
dict_step = [
    (
        "polynomial",
        TSCPolynomialFeatures(degree=3),
    )
]

edmd_poly = EDMD(dict_steps=dict_step, include_id_state=True).fit(X=X_tsc)
edmd_poly_values = edmd_poly.predict(
    X_tsc.initial_states(), time_values=X_tsc.time_values()
)

### Analyze the dictionary

Before we compare the model's time series data to the training data, we investigate how we to analyze the actual process of dictionary transformations in an `EDMD` model.  

This is useful if we are interested and want to investigate the values of the "dictionary space", i.e. the data representation after the transformations were applied to the original data and before it is passed to the final DMD model. To accomblish this we can use the `transform` method of `EDMD`, which only applies the dictionary transformations without processing it through the final estimator. 

In the following cell, we see that the result is a `TSCDataFrame`, which includes the original states "x1" and "x2" plus the generated polynomial features. 

The single dictionary models are accessible with the specified name via `named_steps`. Here, we access the model and its attribute `TSCPolynomialFeatures.powers_` through the `EDMD` model.

In [ ]:
# access models in the dictionary, the name was given in "dict_step" above
print(edmd_poly.named_steps["polynomial"])

print("")
print("polynomial degrees for data (first column 'x1' and second 'x2'):")
print(edmd_poly.named_steps["polynomial"].powers_)

print("")
print("Dictionary space values:")
edmd_poly.transform(X_tsc)

### Compare EDMD with polynomial features with training data

We see that reconstruction of time series improved and the phase portrait now looks a lot better than the previous DMD approach. However, there are still obvious differences and some time series even cross, which is not a behaviour of the original system.

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(14, 5))
for _id, df in X_tsc.itertimeseries():
    ax[0].plot(df["x1"].to_numpy(), df["x2"].to_numpy(), 0.1, c="black")
    include_arrow(ax[0], df)

ax[0].set_title("training data used during fit")
ax[0].set_xlabel("x1")
ax[0].set_ylabel("x2")
ax[0].axis("equal")

for _id, df in edmd_poly_values.itertimeseries():
    ax[1].plot(df["x1"].to_numpy(), df["x2"].to_numpy(), 0.1, c="black")
    include_arrow(ax[1], df)

ax[1].set_title("EDMD with polyomial dictionary")
ax[1].set_xlabel("x1")
ax[1].set_ylabel("x2")
ax[1].axis("equal");

## 3. EDMD: Radial basis function dictionary

In our last attempt, we set up a dictionary with `TSCRadialBasis`. The transform class computes coefficients of each time series sample to a set of radial basis functions, which centres' are distributed on the phase space. The radial basis functions, therefore, provide a way to linearize the phase space's manifold. Here we choose a Gaussian kernel and set the centre of the functions to the initial condition states.

In the time series in "dictionary space," we see that the feature dimension is now much greater than at the beginning (i.e. we provide a larger set of observables to compute the Koopman operator).

In [ ]:
dict_step = [
    (
        "rbf",
        TSCRadialBasis(
            kernel=GaussianKernel(epsilon=0.17), center_type="initial_condition"
        ),
    )
]

# Note that the "extended" part is in the transformations
edmd_rbf = EDMD(dict_steps=dict_step, include_id_state=True).fit(X=X_tsc)
edmd_rbf_values = edmd_rbf.predict(
    X_tsc.initial_states(), time_values=X_tsc.time_values()
)

len_koopman_matrix = len(edmd_rbf.named_steps["dmd"].eigenvectors_right_)
print(f"shape of Koopman matrix: {len_koopman_matrix} x {len_koopman_matrix}")
edmd_rbf.transform(X_tsc)

### Compare EDMD with RBF dictionary with training data and out-of-sample prediction

Again for comparison, we plot the training time series next to the EDMD model's time series. This time the phase portraits match quite well. However, at this stage, this is only an indicator of a successful model. Like for all data-driven machine learning models, there is always the danger of overfitting the training data. A consequence would be a poor generalization for out-of-sample initial conditions. 

The right way to tackle overfitting is to apply cross-validation. For the `EDMD` model this can be achieved with `EDMDCV`, which allows an exhaustive search over a grid of the model's and the dictionary model parameters. *datafold* provides time series splitting for cross-validation which enables measuring the model's quality on unseen (partial) time series data.

In this tutorial, we only add a single out-of-sample initial condition and compare it to the ODE system for a longer time series as in the training data. We used this plot to visually "optimize" the Gaussian kernel epsilon value. If we now predict the time series we want to highlight that the `EDMD` model interpolates in time. This means we are now able to freely choose the time interval and number of time samples at which to evaluate the model. In the time series we can see that the model follows the ground truth solution fairly well for some time. However, the `EDMD` model won't stay on the attractor for $t \rightarrow \infty$ yet.

The problem of overfitting can be seen if `epsilon=1` is set in the Gaussian kernel. The reconstruction phase portrait looks equally well, but the out-of-sample quality decreases. 

In [ ]:
f, ax = plt.subplots(1, 2, sharey=True, figsize=(14, 5))
for _id, df in X_tsc.itertimeseries():
    ax[0].plot(df["x1"].to_numpy(), df["x2"].to_numpy(), 0.1, c="black")
    include_arrow(ax[0], df)

ax[0].set_title("training data used during fit")
ax[0].set_xlabel("x1")
ax[0].set_ylabel("x2")
ax[0].axis("equal")
ax[0].grid()

for _id, df in edmd_rbf_values.itertimeseries():
    ax[1].plot(df["x1"].to_numpy(), df["x2"].to_numpy(), 0.1, c="black")
    include_arrow(ax[1], df)

ax[1].set_title("EDMD with RBF dictionary")
ax[1].set_xlabel("x1")
ax[1].set_ylabel("x2")
ax[1].axis("equal")
ax[1].grid()


# make out-of-sample prediction
X_ic_oos = np.array([[2, 1]])
time_values_oos = np.linspace(0, 7, 400)

ground_truth = system.predict(X_ic_oos, time_values=time_values_oos)
predicted = edmd_rbf.predict(X_ic_oos, time_values=time_values_oos)

f, ax = plt.subplots(figsize=(5, 5))

ax.plot(
    ground_truth.loc[:, "x1"].to_numpy(),
    ground_truth.loc[:, "x2"].to_numpy(),
    label="true system",
)
include_arrow(ax, ground_truth)
ax.plot(
    predicted.loc[:, "x1"].to_numpy(),
    predicted.loc[:, "x2"].to_numpy(),
    c="orange",
    label="edmd_rbf",
)

ax.set_title("out-of-sample prediction")
ax.axis("equal")
ax.grid()
ax.legend();